<a href="https://colab.research.google.com/github/nightted/ML-LeeHongYi-HW/blob/master/HW4_RNN_sentence_classifer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id '1lz0Wtwxsh5YCPdqQ3E3l_nbfJT1N13V8' --output data.zip
!unzip data.zip
!ls

Downloading...
From: https://drive.google.com/uc?id=1lz0Wtwxsh5YCPdqQ3E3l_nbfJT1N13V8
To: /content/data.zip
45.1MB [00:00, 109MB/s]
Archive:  data.zip
replace training_label.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: training_label.txt      
  inflating: testing_data.txt        
  inflating: training_nolabel.txt    
data.zip  sample_data	    training_label.txt
drive	  testing_data.txt  training_nolabel.txt


# Data pre-processing

In [ ]:
import torch.nn as nn
import torch 
import numpy as np
import pandas as pd
import re
from gensim.models import word2vec
 
# Data path
path_train_l = '/content/training_label.txt'
path_train_ul = '/content/training_nolabel.txt'
path_train_t = '/content/testing_data.txt'
w2v_path = '/content/drive/My Drive/Colab Notebooks/W2V_model.model'
 
class PreProcessData():
 
  def __init__(self,path_train_l,path_train_ul,path_train_t,w2v_path=w2v_path):
 
    self.path_train_l = path_train_l
    self.path_train_ul = path_train_ul
    self.path_train_t = path_train_t
    self.w2v_path = w2v_path
    self.W2V_model = None
 
    # Loading the data 
    self.train_label_x , self.train_label_y = self.read_train_data(path_train_l)
    self.train_unlabel_x =  self.read_train_data(path_train_ul)
    self.test_x = self.read_train_data(path_train_t)
 
    # Check the w2v model exsit or not , if not , keep training .
    if  self.w2v_path != None :  
      self.W2V_model = word2vec.Word2Vec.load(self.w2v_path)
    else :
      self.Training_word2vec()
      self.w2v_path = './W2V_model.model'
      self.W2V_model = word2vec.Word2Vec.load(self.w2v_path)
 
    # Extracting the word list and embedding ,matrix
    self.wordtoidx , self.idxtoword , self.Embedding_matrix = self.Preprocess_extractmatrix(self.W2V_model)
    print("!!!Data processing finish!!!")
 
  #fix the abbreviation like was't or i've ....
  def fix_the_abbreviation(self,data):
    
    for ele in data:
      if ele == "'":
        idx = data.index(ele)
        data[idx-1:idx+2] = [''.join(data[idx-1:idx+2])]  
    return data
 
  # loading and pre-processing data
  def read_train_data(self,path,train_label = False,test_label=False):
 
    train_label = True if path.split('.')[0].split('_')[1] == 'label' else False
    test_label = True if path.split('.')[0].split('_')[1] == 'data' else False
 
    if train_label:
 
      with open(path) as f :
        dataX = [self.fix_the_abbreviation(line.strip().split(' ')[2:]) for line in f ]
        f.seek(0) # https://stackoverflow.com/questions/3906137/why-cant-i-call-read-twice-on-an-open-file
        dataY = [line.strip().split(' ')[0] for line in f ]
 
      return dataX ,dataY
    
    elif test_label :
 
      with open(path) as f :
        next(f)
        dataX = [self.fix_the_abbreviation(line.strip()[2:].split(' ')) for line in f ] 
 
      return dataX
 
    else:
 
      with open(path) as f :
        dataX = [self.fix_the_abbreviation(line.strip().split(' ')) for line in f ] 
      return dataX
  
  #Training W2V model 
  def Training_word2vec(self,Total_TrainingData=None):
 
    if Total_TrainingData == None:
      Total_TrainingData = self.train_label_x + self.train_unlabel_x
 
    # W2V parameters
    size = 250 # the dimension of embedding layer
    window = 5  # the window to grab the word before & after the center word
    min_count = 5 # Only collect the words appears > min_count  
    workers = 12 # multi-thread in training
    iter = 10 # the number of epoch , is sensitive to the result?
    sg = 1 # 0: CBOW , 1: Skip-Gram
    hs = 0 # 0: Negative sampling(Logistic) ,1: Softmax
    negative = 4 # the sample amount in negative sampling
 
    print("Training start!!!!!")
    W2V_model = word2vec.Word2Vec(Total_TrainingData , size = size , negative = negative , window = window , min_count = min_count, workers = workers , iter = iter , sg = sg, hs = hs) 
    W2V_model.save('./W2V_model.model')
    print("Training Finish!!!!!")
 
  #Extract the word list and word embedding matrix 
  def Preprocess_extractmatrix(self,model):
 
    wordtoidx = {}
    idxtoword = []
    Embedding_matrix = []
 
    for i , word in enumerate(model.wv.vocab):   
 
      wordtoidx[word] = i
      idxtoword.append(word)
      Embedding_matrix.append(model[word])
    
    Embedding_matrix = torch.tensor(Embedding_matrix) # transfer the embed matrix to tensor in torch
    Result = self.Add_word(model= model,wordtoidx=wordtoidx,idxtoword=idxtoword,Embedding_matrix=Embedding_matrix,added_words=['<UNK>','<PAD>']) 
 
    return Result
 
  # Adding the <unknown> , <Padding> word into the word library of embedding 
  def Add_word(self,model,wordtoidx,idxtoword,Embedding_matrix,added_words):
 
      # add <UNK> and <PAD> into sequence
    for added_word in added_words:
 
      wordtoidx[added_word] = len(wordtoidx)
      idxtoword.append(added_word)
      
      #Adding the <PAD> <UNK> to embedding matrix
      Embedding_dim = model.vector_size
      cat_vector = torch.empty(1,Embedding_dim)
      torch.nn.init.normal_(cat_vector) 
      Embedding_matrix = torch.cat((Embedding_matrix,cat_vector),0)
 
    return wordtoidx , idxtoword ,Embedding_matrix
 
  #Transfer the words in sentence into word vector
  def sen_to_vec(self,data):
 
    all_sen = []
    for i , sen in enumerate(data):
      #print('We are now dealing sentence {}'.format(i))
      sub_sen = []
      for word in sen:
        if word not in self.wordtoidx:
          sub_sen.append(self.wordtoidx['<UNK>'])
        else:
          sub_sen.append(self.wordtoidx[word])
 
      all_sen.append(sub_sen)
    all_sen = self.pad_sen_blank(all_sen)
 
    return torch.LongTensor(all_sen) #training data 記得轉換成 long tensor type!!!!!!!!!!!!!!!!!!!!!!!!!!!
 
  # Cut the word whose length of sentence is larger than 20 , fill the blank to <PAD> for whose length of sentence is less than 20
  def pad_sen_blank(self,all_sen,standard_len = 15):
 
    for sen in all_sen:
      
      if len(sen) >= standard_len:
        sen[:] = sen[:standard_len]
      else:
        sen[standard_len:] =  [self.wordtoidx["<PAD>"]] * (standard_len - len(sen))
    
    return all_sen
  
  #Return the Word vect data
  def Return_data(self):
 
    VecData_label_x = self.sen_to_vec(self.train_label_x)
    VecData_unlabel_x = self.sen_to_vec(self.train_unlabel_x)
    VecData_test_x = self.sen_to_vec(self.test_x)
    Label_y = torch.LongTensor([int(labels) for labels in self.train_label_y])
 
    return VecData_label_x , VecData_unlabel_x , VecData_test_x  , Label_y

In [ ]:
Preprocess = PreProcessData(path_train_l,path_train_ul,path_train_t ,w2v_path)

Training start!!!!!


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Training Finish!!!!!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:111: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


!!!Data processing finish!!!


In [ ]:
VecData_label_x , VecData_unlabel_x , VecData_test_x , Label_y  = Preprocess.Return_data()

In [ ]:
Embedding_Matrix = Preprocess.Embedding_matrix
Embedding_Matrix.shape

torch.Size([52483, 250])

In [ ]:
from torch.utils import data
 
class RNN_dataset(data.Dataset):
 
  def __init__(self,X,Y=None):
 
    self.x = X
    self.y = Y
  
  def __getitem__(self,idx):
 
    if self.y == None:
      return self.x[idx]
 
    else : 
      return self.x[idx] , self.y[idx]
    
  def __len__(self):
    
    return len(self.x)

In [ ]:
import torch.nn as nn
import torch

In [ ]:
#Define LSTM Neural Network
 
class LSTM(nn.Module):
 
  def __init__(self, embedding_dim , hidden_dim , embedding_matrix ,num_layers , dropout):
    super(LSTM,self).__init__()
    # Embedding layer
    self.embedding_matrix = embedding_matrix  # Note that this is type of TORCH TENSOR !!!!!!
    self.embedding = nn.Embedding(self.embedding_matrix.shape[0],self.embedding_matrix.shape[1]) # initialize embedding function
    self.embedding.weight = nn.Parameter(self.embedding_matrix) # Loading the parameter from embedding matrix
    self.embedding.weight.requires_grad = False # fix the params of embedding layer in the network
    #LSTM layer
    self.lstm = nn.LSTM(embedding_dim , hidden_dim ,num_layers ,batch_first = True)
    #Classifier layer
    self.classifier = nn.Sequential( nn.Dropout(dropout),
                                     nn.Linear(hidden_dim,1),
                                     nn.Sigmoid())
    
  def forward(self, x ):
 
    word_embed = self.embedding(x)
    hidden , h_c = self.lstm(word_embed, None)
    output = self.classifier(h_c[0])  #Extract the last hidden layer output
 
    return output

In [ ]:
# Training
 
import torch 
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F
 
#Important!!!!!!!!!! Set Cuda !!!!!!!!!!!!!!!!!
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model params
batch_size = 128
lr = 0.001
num_workers = 8
epoch = 20
 
#split train , validate data
Tx,Ty = VecData_label_x[20000:] , Label_y[20000:]
Vx,Vy = VecData_label_x[:20000] , Label_y[:20000]
 
Train_set = RNN_dataset(Tx,Ty)
Validate_set= RNN_dataset(Vx,Vy)
Trainloader = data.DataLoader(Train_set, batch_size = batch_size, shuffle = True )
ValidateLoader =  data.DataLoader(Validate_set, batch_size = batch_size, shuffle = True  )
 
model = LSTM(embedding_dim =250 , hidden_dim = 150 , embedding_matrix = Embedding_Matrix ,num_layers = 1, dropout = 0.5)
model.cuda()
#別把model.train放這邊ㄚㄚㄚ!!! 要放 train loop 裡面, 因為裡面會不斷在 eval() 跟 train() 之間轉換!!!
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(),lr = lr)

Train_loss = []
Train_acc = []
Validate_loss = []
Validate_acc = []
# training step
for epochs in range(epoch):
 
  model.train()
 
  train_loss , train_acc = 0 , 0
  for times , datas in enumerate(Trainloader):
    ###################################################
    data_x = datas[0].to(device , dtype = torch.long)
    data_y = datas[1].to(device , dtype = torch.float)
    ###################################################
    pred_y = model(data_x)
    pred_y = pred_y.squeeze()
    loss = criterion(pred_y,data_y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
 
  vali_loss , vali_acc = 0 , 0  
  #evaluate the validate data 
  model.eval()
  with torch.no_grad():
 
    for times , train_datas in enumerate(Trainloader):
 
      data_x = train_datas[0].to(device , dtype = torch.long)
      data_y = train_datas[1].to(device , dtype = torch.float)
      pred_y = model(data_x)
      pred_y = pred_y.squeeze() # 去除掉dim =1的dimension
      loss = criterion(pred_y,data_y)   
      train_loss += loss
      train_acc += np.sum(np.around(pred_y.cpu().numpy()) == data_y.cpu().numpy())
      #print(train_loss,train_acc,(times+1)*128)
 
    Train_loss.append(train_loss/Train_set.__len__())
    Train_acc.append(train_acc/Train_set.__len__()*100)
    
 
    for times , vali_datas in enumerate(ValidateLoader):
 
      data_x = vali_datas[0].to(device , dtype = torch.long)
      data_y = vali_datas[1].to(device , dtype = torch.float)
      pred_y = model(data_x)
      pred_y = pred_y.squeeze()
      loss = criterion(pred_y,data_y)   
      vali_loss += loss
      vali_acc += np.sum(np.around(pred_y.cpu().numpy()) == data_y.cpu().numpy())
      #print(vali_loss,vali_acc,(times+1)*128)
 
    Validate_loss.append(vali_loss/Validate_set.__len__())
    Validate_acc.append(vali_acc/Validate_set.__len__()*100)
    print("In EPOCH {} :".format(epochs),"Train_loss:",Train_loss[-1],"Train_acc:",Train_acc[-1],"Vali_loss:",Validate_loss[-1],"Vali_acc:",Validate_acc[-1],"%")

In EPOCH 0 : Train_loss: tensor(0.0036, device='cuda:0') Train_acc: 78.32277777777777 Vali_loss: tensor(0.0036, device='cuda:0') Vali_acc: 77.815 %
In EPOCH 1 : Train_loss: tensor(0.0035, device='cuda:0') Train_acc: 78.7761111111111 Vali_loss: tensor(0.0035, device='cuda:0') Vali_acc: 78.05 %
In EPOCH 2 : Train_loss: tensor(0.0033, device='cuda:0') Train_acc: 80.2461111111111 Vali_loss: tensor(0.0034, device='cuda:0') Vali_acc: 79.38 %
In EPOCH 3 : Train_loss: tensor(0.0032, device='cuda:0') Train_acc: 80.77888888888889 Vali_loss: tensor(0.0034, device='cuda:0') Vali_acc: 79.21000000000001 %
In EPOCH 4 : Train_loss: tensor(0.0031, device='cuda:0') Train_acc: 81.68444444444445 Vali_loss: tensor(0.0034, device='cuda:0') Vali_acc: 79.845 %
In EPOCH 5 : Train_loss: tensor(0.0030, device='cuda:0') Train_acc: 82.46222222222222 Vali_loss: tensor(0.0034, device='cuda:0') Vali_acc: 80.01 %
In EPOCH 6 : Train_loss: tensor(0.0029, device='cuda:0') Train_acc: 83.17722222222223 Vali_loss: tensor(0.

# 總結目前遇到的幾個問題與結論:
1. data type LongTensor 與一般 tensor 差異在哪? 何時需要轉換成 LongTensor ??
2. 在 evaluation 的時候 Trainloader 會有找不到的問題??
3. model.train() 的位置要放對 , 因為eval()也在Loop內,故model.train()也需要在Loop內
4. 盡量在第一行就定義 device = torch.device("cuda" if torch.cuda.is_available() else "cpu") , 後續再用 data.to(device) transfer to cuda

In [ ]:
 class Robot():
 
    __counter = 0
 
    
    def __init__(self):
        self.foo()
 
    def boo(self):
        print('boo')
 
    @classmethod
    def foo(cls):
        Robot.__counter+=1
        print('foo!',Robot.__counter)
 
    @classmethod
    def RobotInstances(cls):
        cls.boo(cls)
        cls.foo()
        return cls
 
Robot.RobotInstances()
x=Robot()
Robot.RobotInstances()

boo
foo! 1
foo! 2
boo
foo! 3


__main__.Robot